In [3]:
import requests
import re
import json
import numpy as np
import pandas as pd

In [4]:
itemList = ['246015818','257954339','141970570','220802331','164592594','183295545','226098709','124461698','227459837','175960813','207484669','191322551','215480646','168990114','210231477','148118543','148118642','139828545','148844538','125368562','132904168','132908159','126708685','202960917','249168748','206174104','184649517','176934669','192027764','231200252','101208279','183451768','190300498','100213976','157136604','157548843']

In [5]:
def apiToData(item):

  #initial call to get the page no.
  item_id = item
  page_no = 0
  page_size = 49

  url = f"https://my.daraz.com.bd/pdp/review/getReviewList?itemId={item_id}&pageSize={page_size}&filter=0&sort=0&pageNo={page_no}"

  req = requests.get(url)
  response = req.json()

  #get data
  totalPages = response['model']['paging']['totalPages']
  data = {}

  for pages in range(totalPages):
    url = f"https://my.daraz.com.bd/pdp/review/getReviewList?itemId={item_id}&pageSize={page_size}&filter=0&sort=0&pageNo={pages}"

    req = requests.get(url)
    data[pages] = req.json()


  # extract comments with rating
  comments = []
  rating = []
  for page in data:
    for review in data[page]['model']['items']:
      comments.append(review["reviewContent"])
      rating.append(review["rating"])


  df = pd.DataFrame({'comments' : comments , 'rating' : rating })
  df = df.drop_duplicates()
  return df

In [6]:
dataset = []

for item in itemList:
  dataset.append(apiToData(item))
df = pd.concat(dataset)
df = df.dropna(axis=0)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7595 entries, 0 to 934
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  7595 non-null   object
 1   rating    7595 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 178.0+ KB


In [8]:
df = df.drop_duplicates()
df['rating'].value_counts()

5    5611
4     503
1     405
3     322
2     173
Name: rating, dtype: int64

In [9]:
df

,comments,rating
0,মোবাইল ডে টু ডে উইজের জন্য অনেক ভাল। কিন্তু বক...,5
1,এটা এতোই ভালো ছিলো যে খুশি হয়ে ২য় আর একটা কিনে...,5
2,ভালোই। তবে এতো দামে sd 680 মানায় না। ক্যামেরা...,3
3,"মোবাইল হাতে পেয়েছি ইনটেক, ভেলিভারি দ্রুত পেয়...",5
4,I took it very well. You can take it if you want.,5
...,...,...
885,thnks,5
918,volo,5
929,bah!,5
932,5/5,5


In [10]:
#only bangla comments

chars_bengali = [chr(c) for c in range(0x0980, 0x09ff)]
df_bengali = df[df.comments.str.contains('|'.join(chars_bengali)).fillna(False)]
df_bengali

,comments,rating
0,মোবাইল ডে টু ডে উইজের জন্য অনেক ভাল। কিন্তু বক...,5
1,এটা এতোই ভালো ছিলো যে খুশি হয়ে ২য় আর একটা কিনে...,5
2,ভালোই। তবে এতো দামে sd 680 মানায় না। ক্যামেরা...,3
3,"মোবাইল হাতে পেয়েছি ইনটেক, ভেলিভারি দ্রুত পেয়...",5
5,"আলহামদুলিল্লাহ, ঠিকঠাক ভাবে ভাবে হাতে পেয়েছি। ...",5
...,...,...
812,চলতি মানের,4
821,ভালো পন্য,5
829,মোটামুটি।,5
844,"আঠা কম,",4


In [11]:
# bangla good
bnGood= df_bengali[df_bengali['rating'] >= 4] 
bnGood.to_csv('bnGood.csv', index=False)
bnGoodCsv = pd.read_csv('bnGood.csv')
bnGoodCsv

,comments,rating
0,মোবাইল ডে টু ডে উইজের জন্য অনেক ভাল। কিন্তু বক...,5
1,এটা এতোই ভালো ছিলো যে খুশি হয়ে ২য় আর একটা কিনে...,5
2,"মোবাইল হাতে পেয়েছি ইনটেক, ভেলিভারি দ্রুত পেয়...",5
3,"আলহামদুলিল্লাহ, ঠিকঠাক ভাবে ভাবে হাতে পেয়েছি। ...",5
4,"ফোন একদম অফিশিয়াল এবং ইনটেক হাতে পেয়েছি,আলহামদ...",5
...,...,...
2406,চলতি মানের,4
2407,ভালো পন্য,5
2408,মোটামুটি।,5
2409,"আঠা কম,",4


In [12]:
# bangla bad
bnBad= df_bengali[df_bengali['rating'] <= 3] 
bnBad.to_csv('bnBad.csv', index=False)
bnBadCsv = pd.read_csv('bnBad.csv')
bnBadCsv

,comments,rating
0,ভালোই। তবে এতো দামে sd 680 মানায় না। ক্যামেরা...,3
1,মনে হচ্ছে ডিসপেলে তে আঠা লাগানো কারন কি? জানতে...,2
2,"৩ মাস ব্যাবহারের পরে রিভিউ দিচ্ছি, দাম হিসাবে ...",2
3,স্পেকের তুলনায় দাম একটু বেশি হলেও ফোনটা খারাপ ...,3
4,IMEI সহ একটা মেমো দিলে ভাল হইত,2
...,...,...
397,একটু পরেই আঠা উঠে যায়😴,2
398,প্যাকেজিং ভাল ছিল না।,3
399,মোটামুটি চলবে।,1
400,নিম্নমানের অনেক,1


In [13]:
#only phonetic column
df_eng = pd.concat([df,df_bengali]).drop_duplicates(keep=False)
df_eng

,comments,rating
4,I took it very well. You can take it if you want.,5
6,Parcel received just now. Packaging was good. ...,5
9,Delivered within 18 hours and the phone is off...,4
10,It's a nice looking phone. faced no problem ye...,4
12,Recommend as it was cheaper than market and al...,5
...,...,...
885,thnks,5
918,volo,5
929,bah!,5
932,5/5,5


In [14]:
# phonetic good
phGood= df_eng[df_eng['rating'] >= 4] 
phGood.to_csv('phoneticGood.csv', index=False)
phGoodCsv = pd.read_csv('phoneticGood.csv')
phGoodCsv

,comments,rating
0,I took it very well. You can take it if you want.,5
1,Parcel received just now. Packaging was good. ...,5
2,Delivered within 18 hours and the phone is off...,4
3,It's a nice looking phone. faced no problem ye...,4
4,Recommend as it was cheaper than market and al...,5
...,...,...
3698,Cool,5
3699,thnks,5
3700,volo,5
3701,bah!,5


In [15]:
# phonetic bad
phBad= df_eng[df_eng['rating'] <= 3] 
phBad.to_csv('phoneticBad.csv', index=False)
phBadCsv = pd.read_csv ('phoneticBad.csv')
phBadCsv

,comments,rating
0,I bought the phone in 26k but now the phone is...,3
1,dui no product dise.keo kinben na. bipod e por...,1
2,Why you are providing bills without IMIE no in...,2
3,Give intact but not given any kinds of money r...,3
4,"GSMArena 8/128 nai, camera valo na.",3
...,...,...
493,Bad quality,1
494,motamoti,2
495,Medium,3
496,Valoi,3
